In [1]:
import numpy as np
import wave
from thinkdsp import read_wave

In [2]:
song1 = 'Ex3_sound5.wav'
song2 = 'Ex3_sound5_embedded.wav'
hidden_message = 'Father Christmas does not exist'

In [3]:
# to load and play the audio file
z = read_wave(song1)
z.make_audio()

## Steps to Encode

In [4]:
# this will read wave audio file
audio_file1 = wave.open(song1, mode = 'rb')

# this will read frames and convert to byte array
frame = bytearray(list(audio_file1.readframes(audio_file1.getnframes())))

# the "hidden" text message
string_secret = hidden_message

# this will append dummy data to fill out rest of the bytes. Receiver shall detect and remove these characters
string_secret = string_secret + int((len(frame)-(len(string_secret)*8*8))/8) *'#'

# this will convert text to bit array
bits = list(map(int, ''.join([bin(ord(i)).lstrip('0b').rjust(8,'0') for i in string_secret])))

# this will replace Leaset Significant Bit (LSB) of each byte of the audio data by one bit from the text bit array
for i, bit in enumerate(bits):
    frame[i] = (frame[i] & 254) | bit
    
# this will retrieve the modified bytes
frame_modified = bytes(frame)

# this will write bytes to a new wave audio file
with wave.open(song2, 'wb') as fd:
    fd.setparams(audio_file1.getparams())
    fd.writeframes(frame_modified)
    
audio_file1.close()

## Steps to Decode. Time to find out the secret message!

In [5]:
audio_file2 = wave.open(song2, mode = 'rb')

# this will convert audio to frames of bytes
frame = bytearray(list(audio_file2.readframes(audio_file2.getnframes())))

# this will extract the Least Significant Bit (LSB) from each byte-frame
lsb = [frame[i] & 1 for i in range(len(frame))]

# this will convert byte stream to string
str = "".join(chr(int("".join(map(str, lsb[i : i + 8])), 2)) for i in range(0, len(lsb), 8))

# this will discard filler symbols
decoded = str.split("###")[0]

# this will print out the hidden message
print("The hidden message is: " + decoded)

audio_file2.close()

The hidden message is: Father Christmas does not exist
